# PROYECTO FINAL GRUPAL  - NBA-StatVision - Lesiones


### Importacion de librerias


In [2]:
import pandas as pd 
import numpy as np 


Importamos dataframe "play_by_play" pero solo leemos la primer fila que es el nombre de la columna


In [3]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_csv = r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\play_by_play.csv'

# df = pd.read_csv(ruta_archivo_csv)

## Leemos solo la primer fila con el nombre de las columnas.
def obtener_nombre_columnas(ruta_archivo_csv):
    
    columnas = pd.read_csv(ruta_archivo_csv, nrows=0).columns.tolist()
    return columnas




Vemos los nombres de las columnas de "play_by_play" (esto es porque nos trae la primer fila del dataset)

In [4]:
primera_fila = obtener_nombre_columnas(ruta_archivo_csv)
# mostramos las columnas:
print("Columnas del archivo 'play_by_play': ")
for col in primera_fila:
    print("-", col)

Columnas del archivo 'play_by_play': 
- game_id
- eventnum
- eventmsgtype
- eventmsgactiontype
- period
- wctimestring
- pctimestring
- homedescription
- neutraldescription
- visitordescription
- score
- scoremargin
- person1type
- player1_id
- player1_name
- player1_team_id
- player1_team_city
- player1_team_nickname
- player1_team_abbreviation
- person2type
- player2_id
- player2_name
- player2_team_id
- player2_team_city
- player2_team_nickname
- player2_team_abbreviation
- person3type
- player3_id
- player3_name
- player3_team_id
- player3_team_city
- player3_team_nickname
- player3_team_abbreviation
- video_available_flag


Este data set no tiene datos de fecha, solo 'game_id', entonces deberemos buscar en otro dataset, si hay game_id con fechas.

# ______________________________________________________________________-

Buscamos en el dataset 'game_info' para ver si contiene una columna de fechas.

In [5]:
## Definimos la ruta donde se encuentra
ruta_game_info = r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\game_info.csv' 

## Leemos el dataset
df = pd.read_csv(ruta_game_info)

## Lo mostramos 
df.head(2000)


,game_id,game_date,attendance,game_time
0,24600001,1946-11-01,NaN,NaN
1,24600003,1946-11-02,NaN,NaN
2,24600002,1946-11-02,NaN,NaN
3,24600004,1946-11-02,NaN,NaN
4,24600005,1946-11-02,NaN,NaN
...,...,...,...,...
1995,25200088,1952-12-05,NaN,NaN
1996,25200090,1952-12-06,NaN,NaN
1997,25200092,1952-12-06,NaN,NaN
1998,25200091,1952-12-06,NaN,NaN


Como tiene columna de fechas, vamos a filtrar los id que estén entre 2010 y 2020

In [7]:
# Cargamos nuevamente el data set, para "parsear" la columna 'game_date'.
df_games = pd.read_csv(ruta_game_info, parse_dates=['game_date'])

# Convertimos al formato fecha.
# df_games['game_date'] = pd.to_datetime(df_games['game_date'])
df_games['game_date'] = pd.to_datetime(df_games['game_date'], format='%Y-%m-%d')

print(df_games['game_date'].dtype)


# Filtrar por fechas:
games_filtred = df_games [
    (df_games['game_date'] >= '2010-10-03') & ## 2010-10-03
    (df_games['game_date'] <= '2020-10-06') ## 2020-10-06
]

# obtener el set de 'game_id' válidos
game_ids_validos = set(games_filtred['game_id'])
print(f'Cantidad de partidos entre 2010 y 2020: {len(game_ids_validos)}')

# Muestro los tipos de datos:
for columna in df_games.columns:
    print(columna)
    print(df_games[columna].describe())
    print('------------------------------------')




datetime64[ns]
Cantidad de partidos entre 2010 y 2020: 11201
game_id
count    5.805300e+04
mean     2.579790e+07
std      6.331314e+06
min      1.050000e+07
25%      2.130028e+07
50%      2.590030e+07
75%      2.880075e+07
max      4.980009e+07
Name: game_id, dtype: float64
------------------------------------
game_date
count                            58053
mean     1995-08-05 03:39:01.646426496
min                1946-11-01 00:00:00
25%                1983-04-24 00:00:00
50%                1998-02-17 00:00:00
75%                2010-11-26 00:00:00
max                2023-06-12 00:00:00
Name: game_date, dtype: object
------------------------------------
attendance
count     52673.000000
mean      15097.689158
std        5142.654175
min           0.000000
25%       12260.000000
50%       16205.000000
75%       18700.000000
max      200049.000000
Name: attendance, dtype: float64
------------------------------------
game_time
count     29942
unique      144
top        2:13
freq       130

Guardo los cambios en el archivo

In [8]:
df_games.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\game_info.csv', index=False)

# _________________________________________________________________________________________

 Uso de Chunksize para filtrar el dataset de 'play_by_play' con los 'game_id' obtenidos de 'game_info'

In [10]:
# Creamos una lista para los Chunks

chunks = []

for chunk in pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\play_by_play.csv', chunksize=100000):
    chunk_filtrado = chunk[chunk['game_id'].isin(game_ids_validos)]
    chunks.append(chunk_filtrado) ## llena la lista 

# Concatenamos los chunks
play_by_play_filtrado = pd.concat(chunks, ignore_index=True)

## mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
print(f'Cantidad de juegos entre 2010 y 2020: {len(play_by_play_filtrado)}')
play_by_play_filtrado.head()





(4564934, 34)
Cantidad de juegos entre 2010 y 2020: 4564934


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,21000003,0,12,0,1,10:52 PM,12:00,NaN,Start of 1st Period (10:52 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,21000003,1,10,0,1,10:52 PM,12:00,Jump Ball Gasol vs. Yao: Tip to World Peace,NaN,NaN,...,Rockets,HOU,4.0,1897,Metta World Peace,1.610613e+09,Los Angeles,Lakers,LAL,0
2,21000003,2,2,52,1,10:53 PM,11:47,MISS Odom 2' Alley Oop Dunk,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
3,21000003,3,4,0,1,10:53 PM,11:44,NaN,NaN,Scola REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,21000003,4,2,42,1,10:53 PM,11:41,NaN,NaN,MISS Brooks 1' Driving Layup,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


Guardamos el dataset en un archivo nuevo de .CSV.
Se denominará 'juego_por_juego.csv'


In [12]:
play_by_play_filtrado.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\juego_por_juego.csv', index=False)
#Se guarda el archivo nuevo ya con los id de juego pertenecientes al rango de fecha que especificamos.


Visualizacion de la tabla juego_por_juego


In [13]:
## mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
play_by_play_filtrado.head(1000)

(4564934, 34)


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,21000003,0,12,0,1,10:52 PM,12:00,NaN,Start of 1st Period (10:52 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,21000003,1,10,0,1,10:52 PM,12:00,Jump Ball Gasol vs. Yao: Tip to World Peace,NaN,NaN,...,Rockets,HOU,4.0,1897,Metta World Peace,1.610613e+09,Los Angeles,Lakers,LAL,0
2,21000003,2,2,52,1,10:53 PM,11:47,MISS Odom 2' Alley Oop Dunk,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
3,21000003,3,4,0,1,10:53 PM,11:44,NaN,NaN,Scola REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,21000003,4,2,42,1,10:53 PM,11:41,NaN,NaN,MISS Brooks 1' Driving Layup,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,21000001,41,2,47,1,7:44 PM,5:42,NaN,NaN,MISS Wade 3' Turnaround Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
996,21000001,42,4,0,1,7:44 PM,5:41,S. O'Neal REBOUND (Off:1 Def:2),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
997,21000001,43,1,5,1,7:44 PM,5:33,Allen 1' Layup (2 PTS) (Rondo 4 AST),NaN,NaN,...,Celtics,BOS,0.0,0,NaN,NaN,NaN,NaN,NaN,0
998,21000001,44,9,1,1,7:44 PM,5:24,NaN,NaN,Heat Timeout: Regular (Reg.1 Short 0),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


Observamos el tipo de datos:

In [14]:
for columna in play_by_play_filtrado.columns:
    print(columna)
    print(play_by_play_filtrado[columna].describe())
    print('------------------------------------')


game_id
count    4.564934e+06
mean     2.240226e+07
std      5.558600e+06
min      1.130000e+07
25%      2.130035e+07
50%      2.150090e+07
75%      2.180019e+07
max      4.190040e+07
Name: game_id, dtype: float64
------------------------------------
eventnum
count    4.564934e+06
mean     2.914173e+02
std      1.763937e+02
min      0.000000e+00
25%      1.410000e+02
50%      2.860000e+02
75%      4.310000e+02
max      1.018000e+03
Name: eventnum, dtype: float64
------------------------------------
eventmsgtype
count    4.564934e+06
mean     3.996416e+00
std      2.685824e+00
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      1.800000e+01
Name: eventmsgtype, dtype: float64
------------------------------------
eventmsgactiontype
count    4.564934e+06
mean     1.214085e+01
std      2.457471e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.100000e+01
max      1.100000e+02
Name: eventmsgactiontype, dtype: float64

 <<< Feli se encarga de lo demas filtro de establa >>>

# ____________________________________________________________

NO VAMOS A USARLO A ESTE DATASET

Abrimos y exploramos el Dataset de Lesiones_2013-2023

In [67]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_lesionados_2013_2023 = r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv'

df_lesionados_1323 = pd.read_csv(ruta_archivo_lesionados_2013_2023)

print(df_lesionados_1323.head())
print("Cantidad de filas y columnas: ", df_lesionados_1323.shape)

   PLAYER_ID    PLAYER_NAME SEASON  SEASON_NUM   AGE PLAYER_HEIGHT_INCHES  \
0    1630639    A.J. Lawson  22-23        22.5  22.0                   78   
1    1631260       AJ Green  22-23        22.5  23.0                   77   
2    1631100     AJ Griffin  22-23        22.5  19.0                   78   
3     203932   Aaron Gordon  22-23        22.5  27.0                   80   
4    1628988  Aaron Holiday  22-23        22.5  26.0                   72   

  PLAYER_WEIGHT  GP   MIN  USG_PCT  ...  AVG_SEC_PER_TOUCH  \
0           179  13   4.8    0.182  ...               1.39   
1           190  32   9.4    0.160  ...               2.61   
2           220  66  19.4    0.174  ...               2.08   
3           235  61  30.1    0.206  ...               2.94   
4           185  57  13.7    0.128  ...               4.17   

   AVG_DRIB_PER_TOUCH  ELBOW_TOUCHES  POST_TOUCHES  PAINT_TOUCHES     TEAM  \
0                0.62            0.0           0.0            0.2      NaN   
1       

Vemos los tipos de datos y  estadisticas de cada columna.

In [43]:
for columna in df_lesionados_1323.columns:
    print(columna)
    print(df_lesionados_1323[columna].describe())
    print('------------------------------------')


PLAYER_ID
count    5.578000e+03
mean     7.789375e+05
std      7.189302e+05
min      7.080000e+02
25%      2.021300e+05
50%      2.035840e+05
75%      1.628399e+06
max      1.641645e+06
Name: PLAYER_ID, dtype: float64
------------------------------------
PLAYER_NAME
count             5578
unique            1389
top       Kemba Walker
freq                25
Name: PLAYER_NAME, dtype: object
------------------------------------
SEASON
count      5578
unique       10
top       21-22
freq        702
Name: SEASON, dtype: object
------------------------------------
SEASON_NUM
count    5578.000000
mean       18.241484
std         2.829290
min        13.500000
25%        15.500000
50%        18.500000
75%        20.500000
max        22.500000
Name: SEASON_NUM, dtype: float64
------------------------------------
AGE
count    5578.000000
mean       26.533883
std         4.253617
min        19.000000
25%        23.000000
50%        26.000000
75%        29.000000
max        43.000000
Name: AGE, dty

Guardamos dataset por si a caso.

In [70]:
df_lesionados_1323.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv', index=False)


# ______________________________________________________________

DATASET QUE VAMOS A USAR

Abrimos el dataset Lesiones_2010-2020


In [45]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_lesionados_2010_2020 = r'C:\Users\Fernando\Desktop\CSV de NBA\injuries_2010-2020.csv'

df_lesionados_1020 = pd.read_csv(ruta_archivo_lesionados_2010_2020)

print(df_lesionados_1020.head())
print("Cantidad de filas y columnas: ", df_lesionados_1020.shape)

         Date     Team Acquired   Relinquished  \
0  2010-10-03    Bulls      NaN  Carlos Boozer   
1  2010-10-06  Pistons      NaN  Jonas Jerebko   
2  2010-10-06  Pistons      NaN  Terrico White   
3  2010-10-08  Blazers      NaN     Jeff Ayres   
4  2010-10-08     Nets      NaN    Troy Murphy   

                                               Notes  
0  fractured bone in right pinky finger (out inde...  
1      torn right Achilles tendon (out indefinitely)  
2  broken fifth metatarsal in right foot (out ind...  
3          torn ACL in right knee (out indefinitely)  
4             strained lower back (out indefinitely)  
Cantidad de filas y columnas:  (27105, 5)


Convertimos los datos de fecha a tipo fecha:

In [46]:
# Convertimos al formato fecha.
df_lesionados_1020['Date'] = pd.to_datetime(df_lesionados_1020['Date'], errors='coerce')
df_lesionados_1020['Date'] = pd.to_datetime(df_lesionados_1020['Date'], format='%y-%m-%d')


print(df_lesionados_1020['Date'].dtype)
print(df_lesionados_1020['Date'].head(10000)) 
print(df_lesionados_1020['Date'].isnull().sum())





datetime64[ns]
0      2010-10-03
1      2010-10-06
2      2010-10-06
3      2010-10-08
4      2010-10-08
          ...    
9995   2014-04-05
9996   2014-04-06
9997   2014-04-06
9998   2014-04-06
9999   2014-04-06
Name: Date, Length: 10000, dtype: datetime64[ns]
0


Vemos los tipos de datos y  estadisticas de cada columna.

In [47]:
for columna in df_lesionados_1020.columns:
    print(columna)
    print(df_lesionados_1020[columna].describe())
    print('------------------------------------')

Date
count                            27105
mean     2015-08-19 17:05:33.503043840
min                2010-10-03 00:00:00
25%                2013-03-17 00:00:00
50%                2015-11-25 00:00:00
75%                2017-12-27 00:00:00
max                2020-10-06 00:00:00
Name: Date, dtype: object
------------------------------------
Team
count     27103
unique       32
top       Spurs
freq       1163
Name: Team, dtype: object
------------------------------------
Acquired
count            9542
unique           1111
top       Dwyane Wade
freq               54
Name: Acquired, dtype: object
------------------------------------
Relinquished
count          17560
unique          1156
top       Kevin Love
freq             101
Name: Relinquished, dtype: object
------------------------------------
Notes
count                 27105
unique                 3114
top       activated from IL
freq                   7342
Name: Notes, dtype: object
------------------------------------


In [48]:
print(df_lesionados_1020.shape)

(27105, 5)


NO LO APLICAMOS

NO REALIZAMOS FILTRO.

Filtramos el dataset y eliminamos las fechas anteriores a 2012-10-30

In [39]:
# Filtrar el DataFrame para que queden solo con las fechas desde el 30-10-2012 en adelante
# df_lesionados_1020_filtrado = df_lesionados_1020[df_lesionados_1020['Date'] > '2012-10-30']

# Mostramos el dataset:
# print(df_lesionados_1020_filtrado.head())
# print(df_lesionados_1020_filtrado.shape)

Guardamos Dataset

In [49]:
df_lesionados_1020.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones.csv', index=False)


# _________________________________________________________________

NO REALIZO UNION DE DATASETS

UNION DE lesiones_1020 y lesiones_1323 en un nuevo dataset, que se llamará: Lesiones

Abrimos ambos datasets.

In [29]:


# Cargar los archivos
# df_2010_2020 = pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2010-2020.csv')
# df_2013_2023 = pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv')


# # Revisamos los nombres de cada una de sus columnas.
# print(df_2010_2020.columns)
# print("\n")
# print(df_2013_2023.columns)

# print(df_2010_2020[['ID']].head())
# print("\n")
# print(df_2013_2023[['PLAYER_ID']].head())






Convertimos los Datos de ID y PLAYER_ID en str, porque puede haber valores no numericos o raro.

In [30]:
# df_2010_2020['ID'] = df_2010_2020['ID'].astype(str)
# df_2013_2023['PLAYER_ID'] = df_2013_2023['PLAYER_ID'].astype(str)



Vamos a unir ambos dataset a travez de ID y PLAYER_ID

In [31]:
# Hacemos el merge por ID de ambos datasets
# how='inner': solo quedarán los registros que tengan coincidencia en ambos datasets.
# Si querés mantener todos los de uno y solo los que coincidan del otro, usá left o right.

# df_unido = pd.merge(
#     df_2010_2020, 
#     df_2013_2023, 
#     left_on='ID', 
#     right_on='PLAYER_ID',
#     how='inner'  # Cambiá a 'outer', 'left' o 'right' según lo que quieras lograr
# )

# #Mostramos
# print(df_unido.shape)
# print(df_unido.head())


Vemos valores no numericos en ID


In [32]:
# mask_no_numericos = ~df_2010_2020['ID'].astype(str).str.isnumeric()
# print(df_2010_2020[mask_no_numericos]['ID'].value_counts())

Vemos los tipos de datos:


In [33]:
# for columna in df_unido.columns:
#     print(columna)
#     print(df_unido[columna].describe())
#     print('------------------------------------')

Guardamos Dataset como Lesiones.csv

In [34]:
# df_unido.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones.csv', index=False)
